In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# df = pd.read_csv('D:\Work\WB\LDT\datasets/digitalization\\220.csv.gz')

In [17]:
pq = pd.read_parquet('D:\Work\WB\LDT\datasets\digitalization\ookla.parquet')
pq

<>:1: SyntaxWarning: invalid escape sequence '\W'
<>:1: SyntaxWarning: invalid escape sequence '\W'
C:\Users\sonle\AppData\Local\Temp\ipykernel_39820\4098743857.py:1: SyntaxWarning: invalid escape sequence '\W'
  pq = pd.read_parquet('D:\Work\WB\LDT\datasets\digitalization\ookla.parquet')


,avg_d_kbps,avg_u_kbps,devices
quadkey_14,,,
221332223300,14668,18683,1
221332223301,10757,2603,3
221332223302,8315,2894,1
223201211213,8953,2588,1
223302003100,17896,18324,1
...,...,...,...
31311202210010,6788,7372,1
31311202210201,11976,3117,1
32110312032211,1047,1136,1


In [3]:
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass

def query_osm_overpass(area_name, structure_name):

    nominatim = Nominatim()
    areaId = nominatim.query(area_name).areaId()
    overpass = Overpass()
    query = overpassQueryBuilder(area=areaId, elementType='way', selector=f'"amenity"="{structure_name}"', out='body', includeGeometry=True)
    result = overpass.query(query, timeout=120)
    
    return result


[overpass] downloading data: [timeout:25][out:json];area(3609088937)->.searchArea;(way["amenity"="hospital"](area.searchArea);); out body geom;


In [4]:
from datetime import datetime
from pyquadkey2 import quadkey as qk

def quarter_start(year: int, q: int) -> datetime:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")

    month = [1, 4, 7, 10]
    return datetime(year, month[q - 1], 1)


def get_tile_url(service_type: str, year: int, q: int) -> str:
    dt = quarter_start(year, q)

    base_url = "https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance"
    url = f"{base_url}/type%3D{service_type}/year%3D{dt:%Y}/quarter%3D{q}/{dt:%Y-%m-%d}_performance_{service_type}_tiles.zip"
    return url

def merge_with_connectivity(out, ookla):
    out = out[~out.lat.isna()]
    out['quadkey_16'] = out.apply(make_quadkey, axis=1)
    out['quadkey_16'] = out['quadkey_16'].astype('str')
    connectivity = ookla[ookla.quadkey.isin(out.quadkey_16)]
    connectivity = connectivity.set_index('quadkey')
    tmp = out.set_index('quadkey_16')
    tmp.index = tmp.index.astype('str')
    tmp = tmp.join(connectivity, how='left')
    tmp = tmp.reset_index().drop_duplicates(
        subset=['name', 'quadkey_16'],
        keep='last').reset_index(drop=True)
    # return connectivity
    return tmp

def make_quadkey(row):
    return qk.from_geo((row['lat'], row['lon']), 16)

def make_details(element):
    tags = element.tags()
    if 'name' in tags:
        out = dict()
        out['amenity'] = tags['amenity']
        out['name'] = tags['name']
        if 'lat' in tags:
            out['lat'] = tags['lat']
            out['lon'] = tags['lon']
        elif bool(element.centerLat()):
            out['lat'] = element.centerLat()
            out['lon'] = element.centerLon()
        elif bool(element.geometry()):
            out['lat'] = element.geometry()['coordinates'][0][0][1]
            out['lon'] = element.geometry()['coordinates'][0][0][0]
        else:
            out['lat'] = None
            out['lon'] = None
        return out
    else:
        return None
    
def filter_elements(result):
    elements = list()
    for element in result.elements():
        elements.append(make_details(element))
    filtered = [element for element in elements if element is not None]
    return filtered

In [5]:
result = query_osm_overpass('Serbia', 'hospital')

filtered = filter_elements(result)
out = pd.DataFrame(filtered)

out

,amenity,name,lat,lon
0,hospital,ГАК Народни фронт,44.809618,20.460292
1,hospital,Неуропсихијатрија,45.768977,19.106787
2,hospital,Клинички центар Војводине,45.254037,19.823653
3,hospital,Специјална болница за реуматске болести,45.249292,19.828312
4,hospital,Клиничко-болнички центар „Звездара“,44.800520,20.501743
...,...,...,...,...
218,hospital,Здравствена станица,43.746802,21.580475
219,hospital,"QKMF ""Safet Idrizi""",42.821678,20.959334
220,hospital,Амбуланта Нови Брачин,43.749266,21.509249
221,hospital,Дом здравља,44.897691,21.421144


In [6]:
out.to_csv('hospitals.csv', index=False)



In [76]:
out = pd.read_csv('D:\Work\WB\LDT\countries\SRB\\raw_data\hospitals.csv')
out.head()

,amenity,name,lat,lon
0,hospital,ГАК Народни фронт,44.809618,20.460292
1,hospital,Неуропсихијатрија,45.768977,19.106787
2,hospital,Клинички центар Војводине,45.254037,19.823653
3,hospital,Специјална болница за реуматске болести,45.249292,19.828312
4,hospital,Клиничко-болнички центар „Звездара“,44.800520,20.501743


In [6]:
import geopandas as gpd

ookla = gpd.read_file('D:\Work\WB\LDT\countries\SRB\\raw_data\\2024-04-01_performance_mobile_tiles\gps_mobile_tiles.shp')
ookla.quadkey = ookla.quadkey.astype('str')

In [102]:
# import geopandas as gpd
# tile_url = get_tile_url("fixed", 2022, 1)
# tiles = gpd.read_file(tile_url)

In [98]:
out_merged = merge_with_connectivity(out, ookla)

In [58]:
# out['quadkey_16'] = out.apply(make_quadkey, axis=1)
# out['quadkey_16'] = out['quadkey_16'].astype('str')
# connectivity = ookla[ookla.quadkey.isin(out.quadkey_16)]
# connectivity = connectivity.set_index('quadkey')
# tmp = out.set_index('quadkey_16')
# tmp.index = tmp.index.astype('str')
# tmp = tmp.join(connectivity, how='left')
# tmp = tmp.reset_index().drop_duplicates(
#         subset=['name', 'quadkey_16'],
#         keep='last').reset_index(drop=True)

In [99]:
len(out_merged[pd.notnull(out_merged.avg_d_kbps)])

156

In [100]:
len(out_merged[pd.isnull(out_merged.avg_d_kbps)])

67

In [101]:
out_merged

,quadkey_16,amenity,name,lat,lon,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry
0,1202331012001322,hospital,ГАК Народни фронт,44.809618,20.460292,70238.0,22595.0,22.0,64.0,26.0,"POLYGON ((20.45654 44.81302, 20.46204 44.81302..."
1,1202312312012332,hospital,Неуропсихијатрија,45.768977,19.106787,61218.0,21440.0,105.0,7.0,3.0,"POLYGON ((19.10522 45.77135, 19.11072 45.77135..."
2,1202313222031202,hospital,Клинички центар Војводине,45.254037,19.823653,41896.0,29210.0,27.0,17.0,14.0,"POLYGON ((19.81934 45.25556, 19.82483 45.25556..."
3,1202313222031221,hospital,Специјална болница за реуматске болести,45.249292,19.828312,14161.0,17566.0,19.0,29.0,8.0,"POLYGON ((19.82483 45.25169, 19.83032 45.25169..."
4,1202331012012120,hospital,Клиничко-болнички центар „Звездара“,44.800520,20.501743,96890.0,39397.0,14.0,6.0,4.0,"POLYGON ((20.50049 44.80133, 20.50598 44.80133..."
...,...,...,...,...,...,...,...,...,...,...,...
218,1202331303031220,hospital,Здравствена станица,43.746802,21.580475,NaN,NaN,NaN,NaN,NaN,None
219,1202333011322313,hospital,"QKMF ""Safet Idrizi""",42.821678,20.959334,31709.0,4135.0,18.0,3.0,3.0,"POLYGON ((20.95642 42.82361, 20.96191 42.82361..."
220,1202331303021213,hospital,Амбуланта Нови Брачин,43.749266,21.509249,85630.0,5962.0,21.0,2.0,2.0,"POLYGON ((21.50574 43.75126, 21.51123 43.75126..."
221,1202331100331013,hospital,Дом здравља,44.897691,21.421144,31445.0,8880.0,20.0,10.0,5.0,"POLYGON ((21.41785 44.89869, 21.42334 44.89869..."


In [8]:
import pandas as pd
df = pd.read_parquet('D:\Work\WB\LDT\countries\SRB\\raw_data\internet\\2022-01-01_performance_fixed_tiles.parquet')

In [10]:
df.drop(['tile', 'tile_x', 'tile_y', 'avg_lat_down_ms', 'avg_lat_up_ms'], axis=1)

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices
0,0022332203013331,50519,24352,18,18,5
1,0022332203013333,10057,3988,9,2,1
2,0022332203031110,900297,75822,7,2,1
3,0022332203031111,370955,62117,8,4,3
4,0022332203031113,323725,37943,23,3,2
...,...,...,...,...,...,...
6780810,3131120300000300,49545,23975,13,4,1
6780811,3200202311023123,26478,108402,570,1,1
6780812,3220103201010202,7110,33881,244,1,1
6780813,3300323003310001,30601,9786,3,1,1


In [17]:
from glob import glob
paths = glob('D:\Work\WB\LDT\countries\SRB\\raw_data\internet\\*_mobile*')

dfs = []
for path in paths:
    print(path)
    df = pd.read_parquet(path)
    df = df.drop(['tile', 'tile_x', 'tile_y', 'avg_lat_down_ms', 'avg_lat_up_ms'], axis=1)
    dfs.append(df)

full = pd.concat(dfs, axis=0).reset_index(drop=True)

D:\Work\WB\LDT\countries\SRB\raw_data\internet\2022-01-01_performance_mobile_tiles.parquet
D:\Work\WB\LDT\countries\SRB\raw_data\internet\2022-04-01_performance_mobile_tiles.parquet
D:\Work\WB\LDT\countries\SRB\raw_data\internet\2022-07-01_performance_mobile_tiles.parquet
D:\Work\WB\LDT\countries\SRB\raw_data\internet\2022-10-01_performance_mobile_tiles.parquet


In [18]:
full

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices
0,0022133222312322,19110,7891,77,3,2
1,0022133222330023,21870,11875,83,2,1
2,0022133222330032,14567,14691,75,9,2
3,0022133222330100,5468,9886,83,1,1
4,0022133222330102,24311,16243,72,1,1
...,...,...,...,...,...,...
15732682,3131120211221222,61559,31302,38,1,1
15732683,3131120211232103,1146,873,49,1,1
15732684,3131120212300111,65504,15991,40,1,1
15732685,3131120212300123,96368,16433,53,1,1


In [19]:
full_gb = full.groupby(['quadkey']).agg({'avg_d_kbps': 'mean',
                                         'avg_u_kbps': 'mean',
                                         'avg_lat_ms': 'mean',
                                         'tests': 'sum', 
                                         'devices': 'sum'}).reset_index()
full_gb


,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices
0,0022133222312322,17660.000000,14932.500000,82.500000,4,3
1,0022133222330023,19547.333333,14359.666667,107.666667,8,3
2,0022133222330031,6790.000000,22363.000000,68.000000,1,1
3,0022133222330032,14094.666667,14110.000000,156.333333,16,5
4,0022133222330100,7543.500000,12821.500000,87.500000,7,2
...,...,...,...,...,...,...
8050644,3131120221002013,54752.000000,26099.000000,59.000000,1,1
8050645,3131120221002122,59200.000000,15101.000000,54.000000,1,1
8050646,3131120221020110,53663.000000,31816.000000,41.000000,2,2
8050647,3131120221022002,29173.000000,30286.000000,62.000000,1,1


In [20]:
full_gb.to_parquet('D:\Work\WB\LDT\countries\SRB\\raw_data\internet\\2022_combined_mobile.parquet', index=False)